In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [14]:
# ensemble csv files
submission_files = ['/opt/ml/level2_objectdetection_cv-level2-cv-17/csv/filtered_swin_pseudo1.csv',
                    '/opt/ml/level2_objectdetection_cv-level2-cv-17/csv/filtered_group0.csv',
                    '/opt/ml/level2_objectdetection_cv-level2-cv-17/csv/filtered_group1.csv',
                    '/opt/ml/level2_objectdetection_cv-level2-cv-17/csv/filtered_group2.csv',
                    '/opt/ml/level2_objectdetection_cv-level2-cv-17/csv/filtered_group3.csv',
                    '/opt/ml/level2_objectdetection_cv-level2-cv-17/csv/filtered_group4.csv',
                    '/opt/ml/level2_objectdetection_cv-level2-cv-17/csv/filtered_swin_384.csv',
                    '/opt/ml/level2_objectdetection_cv-level2-cv-17/csv/filtered_swin_mstrain.csv',
                    ]
submission_df = [pd.read_csv(file) for file in submission_files]

In [15]:
image_ids = submission_df[0]['image_id'].tolist()

In [16]:
# ensemble 할 file의 image 정보를 불러오기 위한 json
annotation = '/opt/ml/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [17]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!
iou_thr = 0.6

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [18]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('/opt/ml/level2_objectdetection_cv-level2-cv-17/preprocessed_swin_ensemble.csv')

submission.head()

,PredictionString,image_id
0,7.0 0.9998800158500671 604.9989013671875 521.3...,test/0000.jpg
1,5.0 0.7429103255271912 130.83419799804688 0.15...,test/0001.jpg
2,1.0 0.9985870718955994 286.2143249511719 306.7...,test/0002.jpg
3,9.0 0.9098024368286133 105.2300033569336 196.7...,test/0003.jpg
4,1.0 0.8340150713920593 200.994384765625 258.41...,test/0004.jpg


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.
